In [44]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from talib import abstract
import math
## This is to Convert a string representation of list into list 
import ast
from functools import partial
import os
from ta.momentum import rsi
from ta.utils import dropna
# import pandas_ta as pta
from collections import defaultdict



In [47]:
files='C:\\Personal-Folders/Stock_projects/rough/importdatafiles/01-07--OCT-2021-WEEKLY-expiry_data__VEGE_NF_AND_BNF_Options.csv'

In [123]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [45]:
def resampler(df,time):
    newdf=pd.DataFrame()
    newdf['Open']=df.Open.resample(time).first()
    newdf['Close']=df.Close.resample(time).last()
    newdf['High']=df.High.resample(time).max()
    newdf['Low']=df.Low.resample(time).min()
    newdf['Tick_Volume']=df['Tick Volume'].resample(time).sum()
    newdf['Volume']=df.Volume.resample(time).sum()
    newdf.dropna(axis=0,inplace=True)
    
    return newdf

In [136]:
df=pd.read_csv(files)
df['Date/Time']=pd.to_datetime(df['Date/Time'].astype(str), format='%d-%m-%Y %H:%M:%S')
df.rename(columns={'Date/Time':'Date','Volume':'Tick Volume','Open Interest':'Volume'},inplace=True)

In [138]:
df.Ticker.unique()

array(['BANKNIFTY', 'BANKNIFTY-FUT', 'BANKNIFTYWK32000PE',
       'BANKNIFTYWK32100PE', 'BANKNIFTYWK32300PE', 'BANKNIFTYWK32400PE',
       'BANKNIFTYWK32500PE', 'BANKNIFTYWK32600PE', 'BANKNIFTYWK32700PE',
       'BANKNIFTYWK32800PE', 'BANKNIFTYWK32900PE', 'BANKNIFTYWK33000PE',
       'BANKNIFTYWK33100PE', 'BANKNIFTYWK33200PE', 'BANKNIFTYWK33300PE',
       'BANKNIFTYWK33400PE', 'BANKNIFTYWK33500CE', 'BANKNIFTYWK33500PE',
       'BANKNIFTYWK33600CE', 'BANKNIFTYWK33600PE', 'BANKNIFTYWK33700PE',
       'BANKNIFTYWK33800CE', 'BANKNIFTYWK33800PE', 'BANKNIFTYWK33900PE',
       'BANKNIFTYWK34000CE', 'BANKNIFTYWK34000PE', 'BANKNIFTYWK34100PE',
       'BANKNIFTYWK34200PE', 'BANKNIFTYWK34300CE', 'BANKNIFTYWK34300PE',
       'BANKNIFTYWK34400PE', 'BANKNIFTYWK34500CE', 'BANKNIFTYWK34500PE',
       'BANKNIFTYWK34600PE', 'BANKNIFTYWK34700CE', 'BANKNIFTYWK34700PE',
       'BANKNIFTYWK34800CE', 'BANKNIFTYWK34800PE', 'BANKNIFTYWK34900CE',
       'BANKNIFTYWK34900PE', 'BANKNIFTYWK35000CE', 'BANKNIFTYWK35

In [181]:
dfo=df.loc[df.Ticker=='NIFTYWK17500CE']
# dfo=df.loc[df.Ticker=='NIFTY']
dfo.set_index('Date',inplace=True)

In [55]:
# #376 min in a trading session.
# datadf[:376].to_csv('DataFiles/niftopdata.csv')

In [193]:
dfo=dfo[-378:].copy()

In [194]:
dfo

,Ticker,Open,High,Low,Close,Tick Volume,Volume
Date,,,,,,,
2021-10-06 15:25:00,NIFTYWK17500CE,153.90,154.55,150.00,150.25,42200,1079650
2021-10-06 15:26:00,NIFTYWK17500CE,150.00,151.00,146.55,147.80,32100,1079650
2021-10-06 15:27:00,NIFTYWK17500CE,147.70,147.95,145.00,145.00,26350,1068800
2021-10-06 15:28:00,NIFTYWK17500CE,144.90,147.75,143.15,143.40,36200,1068800
2021-10-06 15:29:00,NIFTYWK17500CE,144.15,148.95,143.80,148.90,36450,1068800
...,...,...,...,...,...,...,...
2021-10-07 15:25:00,NIFTYWK17500CE,291.40,292.65,291.05,292.65,5000,739000
2021-10-07 15:26:00,NIFTYWK17500CE,292.35,292.35,291.80,292.15,4400,739000
2021-10-07 15:27:00,NIFTYWK17500CE,291.90,292.00,291.15,291.15,6050,735600


In [125]:
def predf(data):
    
    ###Base Time Frame########
    datadf=data.copy()
    # datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'5min')
    supertrendBTF=Supertrend(datadf,10,1)
    supertrendBTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
    supertrendBTF.reset_index(inplace=True)

    ###High Time Frame#########
    datadf=data.copy()
    # datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'15min')
    supertrendHTF=Supertrend(datadf,10,0.8)
    supertrendHTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
    supertrendHTF.reset_index(inplace=True)
    
    #######Merge Base and HTF indicators###
    st=pd.merge(supertrendBTF,supertrendHTF,on='Date',how='left')
    st=st.replace(to_replace=np.nan,method='ffill')
    st=st.set_index('Date')
    
    #########Join everything in single df#######
    datadf=data.copy()
    # datadf.set_index('Date',inplace=True)
    datadf=resampler(datadf,'5min')
    datadf=datadf.join(st)
    # print(datadf.head())
    
    return datadf

In [195]:
staticdf=predf(dfo)

In [214]:
staticdf[-7:-2]

,Open,Close,High,Low,Tick_Volume,Volume,Supertrend_x,Final Lowerband_x,Final Upperband_x,rsi_x,rsima_x,Supertrend_y,Final Lowerband_y,Final Upperband_y,rsi_y,rsima_y
Date,,,,,,,,,,,,,,,,
2021-10-07 14:55:00,314.45,318.35,322.25,310.90,33700,4242200,False,340.02975,323.027795,37.391692,43.228215,False,337.098022,330.801847,44.302195,67.241238
2021-10-07 15:00:00,317.80,294.00,318.90,291.10,43450,4242350,False,340.02975,320.751827,23.811532,40.411313,False,337.098022,324.506853,34.046730,63.158601
2021-10-07 15:05:00,295.10,299.10,305.20,294.05,36000,4226450,False,340.02975,314.921413,29.905199,38.002850,False,337.098022,324.506853,34.046730,63.158601
2021-10-07 15:10:00,297.75,293.85,297.80,291.60,92400,4054250,False,340.02975,309.216416,27.333253,35.861043,False,337.098022,324.506853,34.046730,63.158601
2021-10-07 15:15:00,292.65,292.50,294.25,288.80,31700,3858350,False,340.02975,305.134401,26.659450,33.493464,False,337.098022,309.412543,33.204141,59.639498


In [207]:
dynamicdf=predf(dfo[:354])

In [217]:
dynamicdf[-4:]

,Open,Close,High,Low,Tick_Volume,Volume,Supertrend_x,Final Lowerband_x,Final Upperband_x,rsi_x,rsima_x,Supertrend_y,Final Lowerband_y,Final Upperband_y,rsi_y,rsima_y
Date,,,,,,,,,,,,,,,,
2021-10-07 14:50:00,312.25,314.00,314.00,302.55,19200,4258150,False,340.02975,323.027795,31.263950,43.617271,False,337.098022,330.801847,44.302195,67.241238
2021-10-07 14:55:00,314.45,318.35,322.25,310.90,33700,4242200,False,340.02975,323.027795,37.391692,43.228215,False,337.098022,330.801847,44.302195,67.241238
2021-10-07 15:00:00,317.80,294.00,318.90,291.10,43450,4242350,False,340.02975,320.751827,23.811532,40.411313,False,337.098022,324.506853,35.366674,63.260136
2021-10-07 15:05:00,295.10,297.80,300.20,294.05,4150,845050,False,340.02975,311.921159,28.446405,37.890636,False,337.098022,324.506853,35.366674,63.260136


In [6]:
import time

In [13]:
time.time()

1647249631.7825234

In [15]:
# Get the epoch
obj = time.gmtime(0)
epoch = time.asctime(obj)
print("epoch is:", epoch)

epoch is: Thu Jan  1 00:00:00 1970


In [21]:
datetime.time(15,0,1)

datetime.time(15, 0, 1)

In [53]:
float(str(datetime.datetime.now().time()).split(':')[2])

55.435918

In [33]:
str(datetime.datetime.now().time()).

'15:04:43.720940'

In [54]:
now=datetime.datetime.now().time()


In [55]:
now

datetime.time(17, 33, 42, 478882)

In [61]:
now >= datetime.time(17,35)

False

In [72]:
str(datetime.datetime.now().date())

'2022-03-14'

In [79]:
n=2732.34

In [77]:
atm=50 * round(int(n)/50)

In [81]:
int(n)

2732

In [17]:
dfo=yf.download('TCS.NS',start='2022-03-14',interval='1m')

[*********************100%***********************]  1 of 1 completed


In [18]:
df=dfo.copy()

In [33]:
df.Close[-13:]

Datetime
2022-03-15 12:30:00+05:30    3639.000000
2022-03-15 12:31:00+05:30    3638.000000
2022-03-15 12:32:00+05:30    3635.199951
2022-03-15 12:33:00+05:30    3636.000000
2022-03-15 12:34:00+05:30    3636.000000
2022-03-15 12:35:00+05:30    3636.000000
2022-03-15 12:36:00+05:30    3636.000000
2022-03-15 12:37:00+05:30    3634.949951
2022-03-15 12:38:00+05:30    3636.000000
2022-03-15 12:39:00+05:30    3635.000000
2022-03-15 12:40:00+05:30    3632.050049
2022-03-15 12:41:00+05:30    3630.899902
2022-03-15 12:42:43+05:30    3634.000000
Name: Close, dtype: float64

In [34]:
df.Close.resample('5min',offset='5T').last()[-3:]

Datetime
2022-03-15 12:30:00+05:30    3636.0
2022-03-15 12:35:00+05:30    3635.0
2022-03-15 12:40:00+05:30    3634.0
Freq: 5T, Name: Close, dtype: float64

In [35]:
df.Close.resample('5min').last()[-3:]

Datetime
2022-03-15 12:30:00+05:30    3636.0
2022-03-15 12:35:00+05:30    3635.0
2022-03-15 12:40:00+05:30    3634.0
Freq: 5T, Name: Close, dtype: float64

In [36]:
from pymongo import MongoClient
dbname='testingdb'
mongo_client=MongoClient()
db=mongo_client[dbname]

In [42]:
pnldict={}
pnldict['datetime'] = str('2022-03-15 13:00:01')
pnldict['pnl'] = 20.00
db['dailypnl'].insert_one((pnldict))
pnldict.clear()